In [2]:
%pip install scikit-learn
%pip install pandas
%pip install ucimlrepo
%pip install matplotlib
%pip install matplotlib.pyplot

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Dinho\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Dinho\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Dinho\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Dinho\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
You should consider upgrading via the 'c:\Users\Dinho\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


#### Vamos primeiramente importar o dataset por meio do scikitlearn

Aqui estamos importando esse dataset e ja obtendo os dataframes do pandas para as entradas x e saidas y

In [101]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=15)


# data (as pandas dataframes)
x = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets

#### Vamos ver se esse dataset possui algum dado faltante

In [102]:
null_rows = x[x.isnull().any(axis=1)]
display(null_rows)

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Marginal_adhesion,Single_epithelial_cell_size,Bare_nuclei,Bland_chromatin,Normal_nucleoli,Mitoses
23,8,4,5,1,2,NaN,7,3,1
40,6,6,6,9,6,NaN,7,8,1
139,1,1,1,1,1,NaN,2,1,1
145,1,1,3,1,2,NaN,2,1,1
158,1,1,2,1,3,NaN,1,1,1
164,5,1,1,1,2,NaN,3,1,1
235,3,1,4,1,2,NaN,3,1,1
249,3,1,1,1,2,NaN,3,1,1
275,3,1,3,1,2,NaN,2,1,1
292,8,8,8,1,2,NaN,6,10,1


#### Como podemos observar, ele possui dados faltantes apenas na coluna Bare_nuclei

#### vamos agora, eliminar os dados faltantes dessas tabelas

In [103]:
null_rows_x = null_rows.index
null_rows_y = y.loc[null_rows_x].index
x = x.drop(index=null_rows_x)
y = y.drop(index=null_rows_y)


In [104]:
y["Class"] = y["Class"].replace({4:0,2:1})

#### Com essas entradas e saidas, nos iremos agora obter os valores de entradas e saidas para o treinamento e teste

In [106]:
import math
#Pegar apenas os casos normais para treino, ainda separando, normalizar os dados, e retornar no formato parecido com o do sklearn
def nsa_train_test_split(X, Y, y_label="Class", self_value=1):

    # X[list(X.columns)] = scaler.fit_transform(X)

    Y_train = Y[Y[y_label] == self_value]
    Y_test = Y[Y[y_label] != self_value]


    X_train = X.loc[Y_train.index]  # Select rows from X corresponding to Y_train indices
    X_test = X.loc[Y_test.index]

    X_test = X_test.iloc[:,1:]
    X_test = X_test.iloc[:,1:]

    X_train = X_train.values
    X_test = X_test.values
    return X_train,X_test,Y_train,Y_test


Removendo as colunas com o maior desvio padrao, segundo o artigo

Mass thickness (CT), cell  size uniformity(CS),  the  cell shapeuniformity (CH),  the  empty nucleolus(BN),and normal nucleolus (NN)

In [107]:

x_colunas_selecionadas = x.iloc[:, [0, 1, 2, 5,7]]


In [108]:
x_colunas_selecionadas

,Clump_thickness,Uniformity_of_cell_size,Uniformity_of_cell_shape,Bare_nuclei,Normal_nucleoli
0,5,1,1,1.0,1
1,5,4,4,10.0,2
2,3,1,1,2.0,1
3,6,8,8,4.0,7
4,4,1,1,1.0,1
...,...,...,...,...,...
694,3,1,1,2.0,1
695,2,1,1,1.0,1
696,5,10,10,3.0,10
697,4,8,6,4.0,6


### Treinando o dataset

In [111]:
X_train,X_test,Y_train,Y_test = nsa_train_test_split(x_colunas_selecionadas,y,self_value=1)


In [112]:
X_train

array([[ 5.,  1.,  1.,  1.,  1.],
       [ 5.,  4.,  4., 10.,  2.],
       [ 3.,  1.,  1.,  2.,  1.],
       ...,
       [ 3.,  1.,  1.,  1.,  1.],
       [ 3.,  1.,  1.,  2.,  1.],
       [ 2.,  1.,  1.,  1.,  1.]])

In [117]:
Y_test.size

239

In [135]:
# detectors = censor(X_train,threshold)

from evalNSA import evalNSA


radius = 0.2
threshold = 0.6352
probes = 100
trials = 100
approach = 2

# Perform execution

ypred = evalNSA(X_train, X_test, approach, radius, threshold, probes, trials, verbose=1,test_size=Y_test.size)


=== Training ===
Detectors generated:  25
Detectors discarded:  0
Number of allocation attempts:  7
Coverage rate:  40
=== Testing ===
Number of self samples detected:  2
Number of nonself samples detected:  237
Accuracy:  0.9916317991631799
Average distance:  3.242494393263702
Most matched tracker:  1


In [90]:
from sklearn.metrics import accuracy_score

accuracy_score(ypred,Y_test)

0.0

In [ ]:

print("Accuracy")